In [4]:
import requests as rq
from io import BytesIO # 파일 시스템에 접근하지 않고 메모리 스트림 다루기 위함
import pandas as pd
import datetime
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# generate 헤더 요청 URL
url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"


# generate 페이로드의 양식 데이터
data = {
    "locale": "ko_KR",
    "mktId": "ALL",
    "share": "1",
    "csvxls_isNo": "false",
    "name": "fileDown",
    "url": "dbms/MDC/STAT/standard/MDCSTAT01901",
}

In [2]:
# 브라우저에서 서버로 보내는 헤더값
# referer: 해당 요청이 어디에서 발생했는지 나타냄 (사이트로의 접근을 알림)
# user-agent: 사용자 소프트웨어의 식별 정보 (Mozilla 정보/버전 + 운영체제 정보 + 렌더링 엔진 정보 + 브라우저)

headers = {
    "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
}

In [6]:
# generate 부분의 헤더에서 Referer과 User-Agent를 따올 수 있음 User-Agent는 모두 동일
# KRX의 다른 정보를 따올 때는 Referer만 바꿔주기
# download payload(요청데이터)와 동일해야함


otp = rq.post(url=url, data=data, headers=headers).text

In [7]:
otp

'F2LVFs8EecOpM4pyCPetde3KBrKKc5lsJ4ATVBLbMrsRtSksuLS7Bnxpl86F7dAOvXfGx9S2U5wgvoxsacATRRtmGtORI4WrGDmruVe6oXtCqUypoW0Lp6SAPP0PhVkgOwGaKVsza3ymVMgwlBIx7BdZx3Xdbl9EHo1GilYd0pGe4yGwVPeiBOlGGEQ9bQRlFGLLXSbn99Iqbwlo2NgpPaiWeIbl8DwUioxfGNE/Es0='

In [8]:
# download의 헤더 요청 URL
down_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"

In [9]:
# 서버로부터 데이터 요청 후 읽어들이기
down_sector = rq.post(url=down_url, data={"code": otp}, headers=headers)
sectors = pd.read_csv(BytesIO(down_sector.content), encoding="EUC-KR")

In [10]:
down_sector

<Response [200]>

In [11]:
sectors.to_csv("../data/sectors.csv")

In [12]:
test = sectors[sectors["한글 종목약명"] == "NAVER"][["표준코드", "단축코드", "상장일"]]
test

,표준코드,단축코드,상장일
88,KR7035420009,035420,2008/11/28


In [13]:
if not test.empty:
    print("표준코드: ", test.iloc[0].iloc[0])
    print("단축코드: ", test.iloc[0].iloc[1])
    print("상장일: ", test.iloc[0].iloc[2])
else:
    print("데이터를 찾을 수 없습니다.")

표준코드:  KR7035420009
단축코드:  035420
상장일:  2008/11/28


In [14]:
# 개별종목 시세 generate 헤더 요청 url 및 payload

url_price = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"

data_price = {
    "locale": "ko_KR",
    "tboxisuCd_finder_stkisu0_1": "035420/NAVER",
    "isuCd": "KR7035420009",
    "isuCd2": "KR7005930003",
    "codeNmisuCd_finder_stkisu0_1": "NAVER",
    "param1isuCd_finder_stkisu0_1": "ALL",
    "strtDd": "20200101",
    "endDd": "20210101",
    "adjStkPrc_check": "Y",
    "adjStkPrc": "2",
    "share": "1",
    "money": "1",
    "csvxls_isNo": "false",
    "name": "fileDown",
    "url": "dbms/MDC/STAT/standard/MDCSTAT01701"
}



In [15]:
headers = {
    "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
}

In [16]:
otp_indv = rq.post(url_price, data_price, headers=headers).text

In [17]:
otp_indv

'F2LVFs8EecOpM4pyCPetdRHuThqCXUYMRbKc8lY8svsRtSksuLS7Bnxpl86F7dAOLBmklqtj8Qc/gCYt5OWSaRSZjie/DPxx05OlpeBxa0zXaSw1MeBk8qJlXEbjT+XnqQqHQsSF0H9fPawXEmvvjLMub5G+g0E8gvOdHF129+Cje1kYPX67+oD6/THZQD4gdlpNK6rYJIRZb/Nt9ZIP9OhCG7Of9Z3j2/rtvm7SmJuKHJ810dsIiNQfI+uJ4SpQqOgmuo344IKdIwxxHl+rVvCdHdFoeeev8aRN4dHdujlalHu72lyxDYXYaWGC2xxTHzAu1fkGCuq6Bo+ljyzRNpMc2pNpI5LKDaZMb5/BXqnrGttv1GwkbNWQpIcp0/5Fa/x7ISGYz5gZ6ErIjbkbkdSVIg2L0o6BETZS477SgcnrGttv1GwkbNWQpIcp0/5F6fIoU1jRm80bRDavvHVkMgm/O6dJvHRUlMwCEZye2uY5oiOCdbtTFb4EBKnV1xmPz8UtQ0hG1i67LiEwWWwdo/bwxCgWsTpP6ZppsewRd3k5Rx1hezQSzRxyFk6LTKL0'

In [18]:
indv_down_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"

# 서버로부터 데이터 요청 후 읽어들이기
indv_down = rq.post(url=indv_down_url, data={"code": otp_indv}, headers=headers)
indv_price = pd.read_csv(BytesIO(indv_down.content), encoding="EUC-KR")

In [19]:
indv_price.head()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,2020/12/30,292500,9500,3.36,285000,293000,284000,1190327,345787662000,48047043037500,164263395
1,2020/12/29,283000,2000,0.71,281500,285000,280500,506090,142909061500,46486540785000,164263395
2,2020/12/28,281000,-1000,-0.35,282000,282500,280000,537848,151110988500,46158013995000,164263395
3,2020/12/24,282000,-2000,-0.70,284500,284500,280000,453095,127855006000,46322277390000,164263395
4,2020/12/23,284000,0,0.00,288500,289000,283000,508019,144709809500,46650804180000,164263395


In [22]:
indv_price.tail()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
243,2020/01/08,183000,-4000,-2.14,182000,183500,179000,820867,149007274500,30160851285000,164813395
244,2020/01/07,187000,6500,3.60,182500,187000,181500,328624,60938161000,30820104865000,164813395
245,2020/01/06,180500,-1000,-0.55,180000,182000,178000,287970,51850594000,29748817797500,164813395
246,2020/01/03,181500,-1000,-0.55,185000,186000,180500,454492,82879082000,29913631192500,164813395
247,2020/01/02,182500,-4000,-2.14,187500,188000,182000,300640,55309555500,30078444587500,164813395


In [24]:
indv_price.sort_values(by="일자", ascending=True)
indv_price.to_csv("../data/stocks/nav_years.csv")

In [40]:
# 원하는 종목명 입력
stock_list = []
while True:
    stock = input("종목 입력(종료하시려면 '종료' 입력): ")
    stock_list.append(stock)
    if stock == "종료":
        stock_list.pop(-1)
        break

#종목코드 추출
std = []
abbr = []
day = []
for i in range(len(stock_list)):
    sector = sectors[sectors['한글 종목약명'] == stock_list[i]][['표준코드', '단축코드', '상장일']]
    if not sector.empty:
        std.append(sector.iloc[0].iloc[0])
        abbr.append(sector.iloc[0].iloc[1])
        day.append(sector.iloc[0].iloc[2])
    else:
        print("데이터를 찾을 수 없습니다.")


stock_list, std, abbr, day

KeyboardInterrupt: Interrupted by user

In [ ]:
for i in range(len(stock_list)):

    print(f'종목: {stock_list[i]}')
    print(f"입력예시: 20201230\n최소 시작 날짜: {day[i]}")

    date_1 = input(f"조회 시작일: ")
    date_2 = input(f"조회 종료일: ")


    url_price = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"

    data_price = {
        "locale": "ko_KR",
        "tboxisuCd_finder_stkisu0_1": f"{abbr[i]}/{stock_list[i]}",
        "isuCd": std[i],
        "isuCd2": "KR7005930003",
        "codeNmisuCd_finder_stkisu0_1": stock_list[i],
        "param1isuCd_finder_stkisu0_1": "ALL",
        "strtDd": date_1,
        "endDd": date_2,
        "adjStkPrc_check": "Y",
        "adjStkPrc": "2",
        "share": "1",
        "money": "1",
        "csvxls_isNo": "false",
        "name": "fileDown",
        "url": "dbms/MDC/STAT/standard/MDCSTAT01701"
    }

    headers = {
    "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
    }

    otp_indv = rq.post(url_price, data_price, headers=headers).text

    indv_down_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"

    indv_down = rq.post(url=indv_down_url, data={"code": otp_indv}, headers=headers)
    indv_price = pd.read_csv(BytesIO(indv_down.content), encoding="EUC-KR").sort_values(by="일자", ascending=True)
    

    indv_price.to_csv(f"../data/stocks/{stock_list[i]}.csv")

종목: 삼성전자
입력예시: 20201230
최소 시작 날짜: 1975/06/11
종목: 에코프로
입력예시: 20201230
최소 시작 날짜: 2007/07/20
종목: 전진건설로봇
입력예시: 20201230
최소 시작 날짜: 2024/08/19
종목: 쏠리드
입력예시: 20201230
최소 시작 날짜: 2005/07/08
